In [ ]:
def residualArea(x,circle,signed_network):
    residual = list(signed_network.neighbors(x))
    for re in residual:
        if re in circle:
            residual.remove(re)
    return residual
        

In [ ]:
# based on euclidean distance
def profSimilarity(nodeId, active_center, active_centers,nodeIds,node_features_df):
    import math
    no_of_features = 100
    nodeId_index = nodeIds.index(nodeId)
    ac_index = active_centers.index(active_center)
    # setting to 1 to avoid division by zero error
    sum = 1
    for i in range(0, no_of_features):
        
        sq_diff = (node_features_df["feature"+str(i+1)][nodeId_index] - node_features_df["feature"+str(i+1)][ac_index])**2
        sum += sq_diff

    # returning inverse as high value means less similarity.
    return 1/math.sqrt(sum)

In [ ]:
def edgeExists(nodeId, active_center,signed_network):
    if signed_network.has_edge(nodeId, active_center) or signed_network.has_edge(active_center, nodeId):
        return 1
    else:
        return 0

In [ ]:
def strengthOfTies(nodeId, active_center,signed_network):
    sum = 0
    for degree in list(dict(signed_network.degree([nodeId, active_center])).values()):
        sum+=degree
    sum -= 1
    return 1/sum

In [ ]:
def degreeCentrality(x,circle,signed_network):
    degree = 0
    for c in circle:
        if signed_network.has_edge(x, c) or signed_network.has_edge(c, x):
            degree+=1
    # error if len(circle) = 1
    if len(circle) ==1:
        deg_cen = degree
    else:
        deg_cen = degree/(len(circle)-1)
    return deg_cen

In [ ]:
def algorithm1(nodeIds,active_centers,K,signed_network,node_features_df):
    social_circles = {}
    nodeIdsWAC = list(set(nodeIds) - set(active_centers))
    for i in range (0, K):
        active_center = active_centers[i]
        social_circles[str(active_center)] = []
    for nodeId in nodeIdsWAC:
        maxS = 0
        # active_center which will be most similar to given node
        selectedAC = -1
        for i in range (0, K):
            active_center = active_centers[i]
            p1 = 0
            p2 = 0
            p3 = 0

            p1 = edgeExists(nodeId, active_center,signed_network)

            if p1 == 1:
                p2 = profSimilarity(nodeId, active_center,active_centers,nodeIds,node_features_df)
                p3 = strengthOfTies(nodeId, active_center,signed_network)

                if maxS < p2 + p3:
                    maxS = p2 + p3
                    selectedAC = active_center
    
        if selectedAC != -1:
            social_circles[str(selectedAC)].append(nodeId)
    return social_circles

In [ ]:
def algorithm2_part1(pop_n,nodeIds,K,signed_network,node_features_df):
    Xi = pop_n
    Cij = algorithm1(nodeIds,Xi,K,signed_network,node_features_df)
    
    obj=0
    for k in range(0,K):
        #initialize k=1 and obj=0
        xi = Xi[k]
        
        residual =  residualArea(xi,Cij[str(xi)],signed_network)
        #deg_cen
        deg_cen_C = degreeCentrality(xi,Cij[str(xi)],signed_network)
        deg_cen_R = degreeCentrality(xi,residual,signed_network)
        
        #prof_sim
        prof_sim_C = 0
        for c in Cij[str(xi)]:
            prof_sim_C+=profSimilarity(c, xi,Xi,nodeIds,node_features_df)
        prof_sim_C = prof_sim_C/len(Cij)
        
        prof_sim_R = 0
        for r in residual:
            prof_sim_R+=profSimilarity(r, xi,Xi,nodeIds,node_features_df)
        if len(residual) != 0:
            prof_sim_R = prof_sim_R/len(residual)
        
        #str_C
        str_C = 0
        for c in Cij[str(xi)]:
            str_C+= strengthOfTies(c, xi,signed_network)
        str_C = str_C/len(Cij)
        
        str_R = 0
        for r in residual:
            str_R+= strengthOfTies(r, xi,signed_network)
        if len(residual) != 0:
             str_R = str_R/len(residual)
       
        
        obj+=deg_cen_C - deg_cen_R + prof_sim_C - prof_sim_R + str_C - str_R
    
    
    return obj/K

In [1]:
def parallel_insider_algorithm2_part2(i,Q_desc,K,nodeIds,signed_network,node_features_df):
    import random
    Q_high = list(filter(lambda x:x[K]>1,Q_desc))
    if len(Q_high) >10:
        Q_high = Q_high[:10]

    #print(Q_high)
    Q1 = random.choice(Q_high)

    #print(Q1,Q2)
    x1 = Q1[:-1]
    x1_fit = Q1[-1]
    Q2 = random.choice(Q_high)
    x2 = Q2[:-1]
    x2_fit = Q2[-1]
    while x2 == x1:
        x2 = random.choice(Q_high)
    #random.seed(0)
    randc_pos = random.randint(1,K)

    x1_new = x1[:randc_pos]+x2[randc_pos:]
    x2_new = x2[:randc_pos]+x1[randc_pos:]

    x1_new_fit = algorithm2_part1(x1_new,nodeIds,K,signed_network,node_features_df)
    x2_new_fit = algorithm2_part1(x2_new,nodeIds,K,signed_network,node_features_df)

    max_fit = max(x1_fit,x2_fit,x1_new_fit,x2_new_fit)
    value="null"
    if(x1_fit == max_fit):
        x = x1
        value="x1"
        #print("x1 is the best")
    elif (x2_fit == max_fit):
        x = x2
        value="x2"
        #print("x2 is the best")
    elif (x1_new_fit == max_fit):
        x = x1_new
        value="x1_new"
        #print("x1 new is the best")
    else:
        x = x2_new
        value="x2_new"
        #print("x2 new is the best")

###################original thought###################################        
#         x1 = Q_desc[i][:-1]
#         x2 = Q_desc[i+1][:-1]

#         #cross over
#         #generate a random int randc_pos
#         random.seed(0)
#         randc_pos = randint(1,K)

#         x1_new = x1[:randc_pos]+x2[randc_pos:]
#         x2_new = x2[:randc_pos]+x1[randc_pos:]
#########################################################
    #mutation
    #generate a random position randm_pos [1,k] and rand_id [1,n]
    randm_pos = random.randint(1,K)
    rand_id = random.choice(nodeIds)

    #x = x1_new
    #x = Q_desc[i]
    #repair chomorosome if values of two alleles of a chromosome occurs
    if rand_id in (x[:randm_pos-1]+x[randm_pos:]):
        x_new =x
    else:
        x_new = x[:randm_pos-1]+[rand_id]+x[randm_pos:]
        if len(x_new) !=7:
            
            print("In x_new i =",i,"random pos ",randm_pos," randc pos ",
                  randc_pos,"x has more ", x,"x takes the value",value,
                  "x2: ", x2, "Q2:", Q2,
                 "x1:", x1)

    x_new_fit = algorithm2_part1(x_new,nodeIds,K,signed_network,node_features_df)
    print("This is x_new_fit:", str(x_new_fit),"This is Q_desc[i][-1]",str(Q_desc[i][-1]))
    if (x_new_fit > Q_desc[i][-1]):
        print("Fit better: i is ",i," ", x_new + [x_new_fit])
        return x_new + [x_new_fit]
    else:
        print("Orgin better: i is ",i," ", Q_desc[i])
        return Q_desc[i]